In [78]:
import pandas as pd
import numpy as np
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
import os 
import json
from langchain_core.runnables import RunnablePassthrough
import csv 
pd.set_option('display.max_colwidth', None)

In [68]:
#'DA367EE20E43D3','DB1D9E08BBEC87','D5A533360225FF','DF3D652A0CE74A','D1EB93A6E54702'
#EncounterKey = 'DA367EE20E43D3'

In [79]:
base_path = "F:/LangChain/data/"
noteConcepts = pd.read_csv(base_path + 'noteConcepts.txt', sep="|")
encounters = pd.read_csv(base_path + 'encounters.txt', sep="|")
orders = pd.read_csv(base_path + 'orders.txt', sep="|")
hospitalAcquiredDx = pd.read_csv(base_path + 'hospitalAcquiredDx.txt', sep="|")
noteText = pd.read_csv(base_path + 'noteText.csv')
presentOnAdmitDx = pd.read_csv(base_path + 'presentOnAdmitDx.txt', sep="|")


In [80]:
#restrict all to only EncounterKey that is present in encounters
noteConcepts = noteConcepts[noteConcepts.EncounterKey.isin(encounters.EncounterKey)]
orders = orders[orders.EncounterKey.isin(encounters.EncounterKey)]
hospitalAcquiredDx = hospitalAcquiredDx[hospitalAcquiredDx.EncounterKey.isin(encounters.EncounterKey)]
noteText = noteText[noteText.EncounterKey.isin(encounters.EncounterKey)]
presentOnAdmitDx = presentOnAdmitDx[presentOnAdmitDx.EncounterKey.isin(encounters.EncounterKey)]

In [63]:
#group orders EncounterKey and pivot by Source
orders = orders.groupby(['EncounterKey', 'Source']).size().unstack(fill_value=0).reset_index()
orders.head()

Source,EncounterKey,*Unspecified,Haiku,Manage Orders,Medication List,Modify from Manage Orders,Modify from Rx Sidebar,Ord Rec Admission New Orders,Ord Rec Admission Reorder PTA,Ord Rec Admission Review PTA,Ord Rec Discharge New Orders,Ord Rec Discharge Review Orders,Ord Rec PreTransfer New Orders,Ord Rec PreTransfer Reorder
0,D009F2D78A0374,0,0,1,0,0,0,0,0,0,0,0,0,0
1,D00F3A8D5F43B2,0,0,2,0,0,0,0,0,0,0,1,0,0
2,D01FCB9EFFD6D5,0,0,1,0,0,0,0,0,0,0,0,0,0
3,D02DC627A3743A,0,0,2,0,0,0,0,1,0,0,0,0,0
4,D03868A75BBE62,0,0,1,0,0,0,0,0,0,0,1,0,0


In [65]:
#filter to where Ord Rec Admission Reorder PTA or Ord Rec Admission Review PTA > 0 and Ord Rec Discharge New Orders and Ord Rec Discharge Review Orders = 0 
orders = orders[(orders['Ord Rec Admission Reorder PTA'] > 0) | (orders['Ord Rec Admission Review PTA'] > 0)]
orders = orders[(orders['Ord Rec Discharge New Orders'] == 0) & (orders['Ord Rec Discharge Review Orders'] == 0)]
orders.head(10)

Source,EncounterKey,*Unspecified,Haiku,Manage Orders,Medication List,Modify from Manage Orders,Modify from Rx Sidebar,Ord Rec Admission New Orders,Ord Rec Admission Reorder PTA,Ord Rec Admission Review PTA,Ord Rec Discharge New Orders,Ord Rec Discharge Review Orders,Ord Rec PreTransfer New Orders,Ord Rec PreTransfer Reorder
3,D02DC627A3743A,0,0,2,0,0,0,0,1,0,0,0,0,0
10,D04BAD30F16577,0,0,0,0,0,0,0,1,0,0,0,0,0
13,D0562973ED5D20,1,0,3,0,1,0,0,1,0,0,0,0,0
16,D068E26FFF9F43,0,0,0,0,0,0,0,1,0,0,0,0,0
32,D0E44FD0BBD96F,0,0,0,0,0,0,0,1,0,0,0,0,0
41,D1257B7E60B8C1,0,0,0,0,0,0,0,1,0,0,0,0,0
43,D136D97DF32DB1,0,0,0,0,0,0,0,1,0,0,0,0,0
48,D17AFAD1D7BB68,0,0,0,0,1,0,0,1,0,0,0,0,0
50,D188F3E1578728,0,0,0,0,0,0,0,1,0,0,0,0,0
60,D1E15B1F941CE4,1,0,0,0,0,0,0,1,0,0,0,0,0


In [15]:
presentOnAdmitDx.head()

,EncounterKey,DeidLds,DiagnosisKey,DiagnosisEpicId,Name,EmergencyCareNeededNotPreventablePercentage,EmergencyCareNeededPreventablePercentage,EmergencyPrimaryCareTreatablePercentage,EmergencyNonEmergentPercentage,EmergencyAlcoholUsePercentage,EmergencyDrugUsePercentage,EmergencyInjuryPercentage,EmergencyPsychPercentage,EmergencyUnclassifiedPercentage
4,D00F3A8D5F43B2,deid,126346,211602,Germ cell tumor (CMS code),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,D00F3A8D5F43B2,deid,196992,71791,Retroperitoneal mass,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52,D03868A75BBE62,deid,-1,*Unspecified,*Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65,D03F9F35A76C47,deid,253241,288271,At risk for opportunistic infections,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66,D03F9F35A76C47,deid,552197,203477,Chronic abdominal pain,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
#find 5 EncounterKeys where there are < 10 notes 
encounterKeys = noteText.groupby('EncounterKey').size().reset_index(name='counts')
#order by counts
encounterKeys = encounterKeys.sort_values(by='counts', ascending=True)
encounterKeys = encounterKeys[encounterKeys['counts'] < 20]
encounterKeys = encounterKeys.sample(5)
encounterKeys = encounterKeys['EncounterKey'].tolist() 

In [29]:
encounterKeys

['D91E787AE2C10E',
 'D58201CEC3CC7B',
 'DB3DD030F1436E',
 'D51A4F043AD4B6',
 'DDDBFC7BE1A7C8']

In [77]:
#find hospitalAcquiredDx that contains the word "ulcer"
ulcer = noteText[noteText['NoteText'].str.contains("Medications Stopped During this", case=False)]
#also filter to where NoteText contains prazole
ulcer = ulcer[ulcer['NoteText'].str.contains("prazole", case=False)]
#filter to where NoteText does not contain "none"
ulcer = ulcer[~ulcer['NoteText'].str.contains("none", case=False)]
#filter to where the NoteText contains "Medications Stopped During this" and then "prazole" in the next 50 characters
ulcer = ulcer[ulcer['NoteText'].str.contains("Medications Stopped During this", case=False) & ulcer['NoteText'].str.contains("prazole", case=False)]
ulcer.tail(10)

deid_note_key    EncounterKey    NoteDate DepartmentSpecialty  \
25459  DD0BA8F1BE92EB  D53CFBE6366F99  2023-09-15   Inpatient Nursing   
25487  DD0E6A047886D2  D4BAEF91CA7588  2023-09-21   Inpatient Nursing   
26121  DD64CF18A30691  D3E9373625B9E7  2023-09-07   Inpatient Nursing   
26776  DDBBE90158CB5A  D23F88442E7B4F  2023-09-14   Inpatient Nursing   
26849  DDC4DAECD02D53  DF015E65B222CE  2023-09-14   Inpatient Nursing   
27055  DDE070E26FF898  D70969A027A062  2023-09-07   Inpatient Nursing   
27230  DDF7698F2E9771  D20658598D8603  2023-09-11   Inpatient Nursing   
27244  DDF8E6F216E051  D74F31983BABBA  2023-09-21   Inpatient Nursing   
28142  DE6F2E55BF22CB  DB99BE31CB292C  2023-09-15   Inpatient Nursing   
30017  DF6443C9068A57  D6EAEC57995F96  2023-09-16   Inpatient Nursing   

      ProviderSpecialty ProviderType  \
25459               NaN   Pharmacist   
25487          Pharmacy   Pharmacist   
26121               NaN   Pharmacist   
26776        Transplant   Pharmacist   
26849          Pharmacy   Pharmacist   
27055               NaN   Pharmacist   
27230          Pharmacy   Pharmacist   
27244               NaN   Pharmacist   
28142        Transplant   Pharmacist   
30017               NaN   Pharmacist   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

In [ ]:
#D6253A5CE371EA- example with gastrointestinal bleed should continue PPI 
#DFAFED1811B871- example with a documented history of a GI bleed should continue PPI
#D4865B8BBB294E- example with GI bleeding that occurred during inpatient stay should continue PPI
#D00F3A8D5F43B2- example with symptoms of heartburn (upper GI symptoms) evaluate for endoscopy to determine if PPI is needed
#D8017C77BA15FA- 14d h pylori treatment if pt treated for 2wks and asymptomatic, stop PPI
#DA15CCF42ACF8B- Barrett's esophagus, continue PPI
#D45728A2EFD315- example with esophagitis, continue PPI
#DFB07E6B8F0957- example with chronic NSAID use and pharmacist deprescribing note to evaluate for PPI
#D1BD3665C06499- example where pt should be on due to history of surgery and chronic GERD symptoms
#D068E26FFF9F43- example where pt came in on and no clear reason to continue
#D0E44FD0BBD96F- example where pt came in on and no clear reason to continue
#D17AFAD1D7BB68- should stop PPI based on Fundoplication being performed while pt was in surgery



In [ ]:
#filter all to encounter_key
noteConcepts = noteConcepts[noteConcepts['EncounterKey'] == EncounterKey]
encounters = encounters[encounters['EncounterKey'] == EncounterKey]
orders = orders[orders['EncounterKey'] == EncounterKey]
hospitalAcquiredDx = hospitalAcquiredDx[hospitalAcquiredDx['EncounterKey'] == EncounterKey]
noteText = noteText[noteText['EncounterKey'] == EncounterKey]